# Number of synapses per connection

- Calculate number of synapses per connection
- Validate number of synapses per connection

<img src="https://raw.githubusercontent.com/antonelepfl/hbp_school/master/nsyns_per_conn.png">

In [ ]:
import itertools

from collections import defaultdict

import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import seaborn

from bluepy.v2 import Cell, Circuit
from connectome_tools.dataset import read_nsyn

In [ ]:
circuit = Circuit('circuit/CircuitConfig')

In [ ]:
nsample = 10

In [ ]:
model_mean, model_std = defaultdict(dict), defaultdict(dict)
for pre_mtype, post_mtype in itertools.product(circuit.cells.mtypes, circuit.cells.mtypes):
    pre_gids = circuit.cells.ids({Cell.MTYPE: pre_mtype, '$target': 'mc2_Column'}, limit=nsample)
    sample = circuit.stats.sample_pathway_synapse_count(nsample, pre=pre_gids, post={Cell.MTYPE:  post_mtype})
    if len(sample) > 0:
        model_mean[post_mtype][pre_mtype] = sample.mean()
        model_std[post_mtype][pre_mtype] = sample.std()
    else:
        model_mean[post_mtype][pre_mtype] = np.nan
        model_std[post_mtype][pre_mtype] = np.nan
        
model_mean, model_std = pd.DataFrame(model_mean), pd.DataFrame(model_std)

In [ ]:
seaborn.heatmap(model_mean);

In [ ]:
bio_path = 'circuit/bioname/nsyn_per_connection_20180125.tsv'

In [ ]:
df = read_nsyn(bio_path).rename(columns={
    'mean': 'bio_mean',
    'std': 'bio_std'
})
df.head()

In [ ]:
df['mod_mean'] = np.NAN
df['mod_std'] = np.NAN

In [ ]:
for idx in df.index:
    pre = df.loc[idx, 'from']
    post = df.loc[idx, 'to']
    df.loc[idx, 'mod_mean'] = model_mean[post][pre]
    df.loc[idx, 'mod_std'] = model_std[post][pre]
df.head()

In [ ]:
x = df['mod_mean'].values
y = df['bio_mean'].values
l = np.linspace(0, max(x.max(), y.max()), 50)
plt.errorbar(x, y, xerr=df['mod_std'].values, yerr=df['bio_std'].values, fmt='o', ecolor='g', capthick=2)
plt.plot(l, l, 'k--')
plt.xlabel('Model (#)')
plt.ylabel('Experiment (#)')
plt.title('Synapses per connection');